<a href="https://colab.research.google.com/github/Cypher29/AI_repo/blob/main/tokenizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import AutoModel

In [ ]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer
from transformers import pipeline
#import anthropic

In [ ]:
hf_token = userdata.get('HUGGINGFACE_KEY')
login(hf_token, add_to_git_credential=True)

In [ ]:
config = AutoModel.from_pretrained('google-bert/bert-base-uncased')

In [ ]:
print(config)

In [ ]:
PHI3_MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
QWEN2_MODEL_NAME = "Qwen/Qwen2-7B-Instruct"
STARCODER2_MODEL_NAME = "bigcode/starcoder2-3b"

In [ ]:
phi3_tokenizer = AutoTokenizer.from_pretrained(PHI3_MODEL_NAME)

text = "I am excited to show Tokenizers in action to my LLM engineers"
print(tokenizer.encode(text))
print()
tokens = phi3_tokenizer.encode(text)
print(phi3_tokenizer.batch_decode(tokens))

In [ ]:
qwen2_tokenizer = AutoTokenizer.from_pretrained(QWEN2_MODEL_NAME)

text = "I am excited to show Tokenizers in action to my LLM engineers"
print(tokenizer.encode(text))
print()
print(phi3_tokenizer.encode(text))
print()
print(qwen2_tokenizer.encode(text))

This is sentiment analysis model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased-finetuned-sst-2-english', trust_remote_code=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased', trust_remote_code=True)

In [ ]:
len(tokenizer)

In [ ]:
print(tokenizer)

In [ ]:
text = "Hello, Explain about software development life cycle"
tokens = tokenizer.encode(text)
vocabsize = tokenizer.vocab_size
print("Vocab size : " ,vocabsize)
tokens

In [ ]:
len(tokens)

In [ ]:
tokenizer.decode(tokens)

In [ ]:
tokenizer.all_special_tokens

In [ ]:
tokenizer.batch_decode(tokens)

In [ ]:
tokenizer.get_added_vocab()

In [ ]:
PHI3_MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

In [ ]:
phi3_tokenizer = AutoTokenizer.from_pretrained(PHI3_MODEL_NAME)

text = "I am excited to show Tokenizers in action to my LLM engineers"
print(tokenizer.encode(text))
print()
tokens = phi3_tokenizer.encode(text)
print(phi3_tokenizer.batch_decode(tokens))

In [ ]:
tokenizer.all_special_tokens
tokenizer.get_vocab()


In [96]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

inputs = tokenizer("My bike has trouble and I feel good as I don't need to got to market ", return_tensors="pt")
print(inputs)
len(inputs)
with torch.no_grad():
    logits = model(**inputs).logits
print(logits)
predicted_class_id = logits.argmax().item()
print(predicted_class_id)
model.config.id2label[predicted_class_id]

{'input_ids': tensor([[ 101, 2026, 7997, 2038, 4390, 1998, 1045, 2514, 2204, 2004, 1045, 2123,
         1005, 1056, 2342, 2000, 2288, 2000, 3006,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[-4.1062,  4.4022]])
1


'POSITIVE'

In [ ]:
from openai import OpenAI
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [91]:
system_message = "You are an assistant that is great at completing sentense. Always be accurate. If you don't know the answer, say so."
#user_prompt = "Plucking apples from apple tree in apestreat. How many p's are there in above sentense"
user_prompt = "food ball is a game"
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [92]:
#gpt-3.5-turbo
#gpt-4.1
completion = openai.chat.completions.create(model='gpt-4.1', messages=prompts, temperature=0.8)
print(completion.choices[0].message.content)

Football is a game played between two teams, where the objective is to score points by getting the ball into the opposing team's goal or end zone, depending on the type of football (such as soccer, American football, or rugby).


In [83]:
#completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
Response = openai.responses.create(model='gpt-4.1',  tools=[{"type": "web_search_preview"}],input="What is today's date")
print(Response.output_text)
print(Response)

#print(completion.choices[0].message.content)

Apr 29, 2025, 6:09:19 AM 
Response(id='resp_6810a54c962881918175460231c67c5e0dceb501030cc726', created_at=1745921356.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFunctionWebSearch(id='ws_6810a54d37cc81918231285022163d570dceb501030cc726', status='completed', type='web_search_call'), ResponseOutputMessage(id='msg_6810a54f0124819180a8e980238385970dceb501030cc726', content=[ResponseOutputText(annotations=[], text='Apr 29, 2025, 6:09:19\u202fAM ', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[WebSearchTool(type='web_search_preview', search_context_size='medium', user_location=UserLocation(type='approximate', city=None, country='US', region=None, timezone=None))], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), service_

In [ ]:
import math
response = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts,
                                        max_tokens=3,   temperature=0.8,  logprobs=True,     seed=42,         top_logprobs=7,        stream=True)
predictions = []
for chunk in response:
            if chunk.choices[0].delta.content:
                token = chunk.choices[0].delta.content
                logprobs = chunk.choices[0].logprobs.content[0].top_logprobs
                logprob_dict = {item.token: item.logprob for item in logprobs}

                # Get top predicted token and probability
                top_token = token
                top_prob = logprob_dict[token]

                # Get alternative predictions
                alternatives = []
                for alt_token, alt_prob in logprob_dict.items():
                    if alt_token != token:
                        alternatives.append((alt_token, math.exp(alt_prob)))
                alternatives.sort(key=lambda x: x[1], reverse=True)

                prediction = {'token': top_token, 'probability': math.exp(top_prob),'alternatives': alternatives[:3]}
                predictions.append(prediction)

print(predictions)